# Analyse exploratoire COVID-19

Ce notebook contient les premières analyses et visualisations des données COVID-19.

## Utilisation

1. Assurez-vous d'avoir les données dans `data/raw/covid_data.csv`
2. Exécutez les cellules pour explorer les données
3. Modifiez et expérimentez selon vos besoins

